# Pinecone

In [1]:
# pip install pinecone-client

In [2]:
# pip install --upgrade -q pinecone-client

In [6]:
pip show pinecone-client

Name: pinecone-client
Version: 4.1.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /Users/walkyz/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages
Requires: certifi, tqdm, typing-extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [9]:
from pinecone import Pinecone
pc = Pinecone() # Expect PINECONE_API_KEY
pc.list_indexes()

{'indexes': []}

## Working with Pinecone Indexes

In [13]:
# Creating an index

In [23]:
from pinecone import ServerlessSpec
index_name = 'langchain'

def create_index(name, region='us-east-1'):
    if name not in pc.list_indexes().names():
        print(f'Creating index: {name}')
        pc.create_index(
            name=name,
            dimension=1536,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region=region
            )
        )
        print('Index created! 😀')
    else:
        print(f'Index {name} already exists!')

create_index(index_name)

Creating index: langchain
Index created! 😀


In [49]:
create_index(index_name)

Creating index: langchain
Index created! 😀


In [25]:
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'langchain-6xbqkm1.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [26]:
# Delete an index
def delete_index(name):
    if name in pc.list_indexes().names():
        print(f'Deleting index {name} ...')
        pc.delete_index(name)
        print('Done')
    else:
        print(f'Index {name} does not exist!')

In [48]:
delete_index(index_name)

Deleting index langchain ...
Done


In [31]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

## Working with vectors

In [50]:
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('abcde')

index_name = 'langchain'
index = pc.Index(index_name)

index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [43]:
# Update a vector
index.upsert(vectors=[('c', [0.5] * 1536)])

{'upserted_count': 1}

In [44]:
# fetching vectors
# index = pc.Index(index_name)
index.fetch(ids=['c', 'd'])

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'c': {'id': 'c',
                   'values': [0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
                              0.5,
             

In [45]:
# Deleting vectors
index.delete(ids=['b', 'c'])

{}

In [51]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

In [47]:
index.fetch(ids=['x'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [52]:
# query
query_vector = [random.random() for _ in range(1536)]

In [54]:
# Find the most similar vectors
index.query(
    vector=query_vector,
    top_k=3,
    include_values=False
)

{'matches': [{'id': 'c', 'score': 0.753353953, 'values': []},
             {'id': 'd', 'score': 0.75001, 'values': []},
             {'id': 'a', 'score': 0.749136806, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

## Pinecone Namespace

In [55]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

In [60]:
index.delete(ids=['a', 'b', 'c', 'd', 'e'])

{}

In [61]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [62]:
index = pc.Index('langchain')

import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
ids = list('abcde')
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [63]:
vectors = [[random.random() for _ in range(1536)] for v in range(3)]
ids = list('xyz')
index.upsert(vectors=zip(ids, vectors), namespace='first-namespace')

{'upserted_count': 3}

In [64]:
vectors = [[random.random() for _ in range(1536)] for v in range(2)]
ids = list('qp')
index.upsert(vectors=zip(ids, vectors), namespace='second-namespace')

{'upserted_count': 2}

In [65]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5},
                'first-namespace': {'vector_count': 3},
                'second-namespace': {'vector_count': 2}},
 'total_vector_count': 10}

In [66]:
index.fetch(ids=['x'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [67]:
index.fetch(ids=['x'], namespace='first-namespace')

{'namespace': 'first-namespace',
 'usage': {'read_units': 1},
 'vectors': {'x': {'id': 'x',
                   'values': [0.269287884,
                              0.507660091,
                              0.838629961,
                              0.109048158,
                              0.485899121,
                              0.91773057,
                              0.335255235,
                              0.764584,
                              0.00648715114,
                              0.768034577,
                              0.148230448,
                              0.590354919,
                              0.773065209,
                              0.631834567,
                              0.195141822,
                              0.821095049,
                              0.876496792,
                              0.983675659,
                              0.163456529,
                              0.662954211,
                              0.761222124,
       

In [68]:
index.delete(ids=['x'], namespace='first-namespace')

{}

In [69]:
index.fetch(ids=['x'], namespace='first-namespace')

{'namespace': 'first-namespace', 'usage': {'read_units': 1}, 'vectors': {}}

In [70]:
index.delete(delete_all=True, namespace='first-namespace')

{}